<a id="introduction"></a>
## Introduction to Dask cuDF
#### By Paul Hendricks
-------

In this notebook, we will show how to work with cuDF DataFrames distributed across multiple GPUs using Dask.

**Table of Contents**

* [Introduction to Dask cuDF](#introduction)
* [Setup](#setup)
* [Dask cuDF Series Basics](#series)
* [Dask cuDF DataFrame Basics](#dataframes)
* [Input/Output](#io)
* [Dask cuDF API](#daskcudfapi)
* [Conclusion](#conclusion)

<a id="setup"></a>
## Setup

This notebook was tested using the following Docker containers:

* `rapidsai/rapidsai-nightly:0.8-cuda10.0-devel-ubuntu18.04-gcc7-py3.7` from [DockerHub - rapidsai/rapidsai-nightly](https://hub.docker.com/r/rapidsai/rapidsai-nightly)

This notebook was run on the NVIDIA Tesla V100 GPU. Please be aware that your system may be different and you may need to modify the code or install packages to run the below examples. 

If you think you have found a bug or an error, please file an issue here: https://github.com/rapidsai/notebooks/issues

Before we begin, let's check out our hardware setup by running the `nvidia-smi` command.

In [3]:
!nvidia-smi

Wed Sep 25 18:10:38 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.26       Driver Version: 430.26       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Quadro GV100        Off  | 00000000:15:00.0 Off |                  Off |
| 29%   42C    P2    25W / 250W |      0MiB / 32508MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Quadro GV100        Off  | 00000000:2D:00.0  On |                  Off |
| 35%   48C    P0    25W / 250W |    200MiB / 32498MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

Next, let's see what CUDA version we have:

In [4]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


#### Creating a Dask cudf DataFrame from Dask DataFrame (coming soon!)

In [5]:
import numpy as np; print('NumPy Version:', np.__version__)
import pandas as pd; print('Pandas Version:', pd.__version__)


pandas_df = pd.DataFrame({'a': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'b': [0.0, 0.1, 0.2, None, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]})
print(pandas_df)

NumPy Version: 1.16.4
Pandas Version: 0.24.2
     a    b
0    0  0.0
1    1  0.1
2    2  0.2
3    3  NaN
4    4  0.4
5    5  0.5
6    6  0.6
7    7  0.7
8    8  0.8
9    9  0.9
10  10  1.0


In [6]:
import dask; print('Dask Version:', dask.__version__)
import dask.dataframe as dd


dask_df = dd.from_pandas(pandas_df, npartitions=8)
dask_df

Dask Version: 2.4.0


,a,b
npartitions=5,,
0,int64,float64
2,...,...
...,...,...
8,...,...
10,...,...


In [7]:
import dask_cudf; print('Dask cuDF Version:', dask_cudf.__version__)


ddf = dask_cudf.from_dask_dataframe(dask_df)
ddf

Dask cuDF Version: 0.10.0a+1601.gd910060a9


,a,b
npartitions=5,,
0,int64,float64
2,...,...
...,...,...
8,...,...
10,...,...


#### Creating a Dask cudf DataFrame from cuDF DataFrame (coming soon!)

In [8]:
import pandas as pd


pandas_df = pd.DataFrame({'a': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'b': [0.0, 0.1, 0.2, None, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]})
print(pandas_df)

     a    b
0    0  0.0
1    1  0.1
2    2  0.2
3    3  NaN
4    4  0.4
5    5  0.5
6    6  0.6
7    7  0.7
8    8  0.8
9    9  0.9
10  10  1.0


In [9]:
import cudf; print('cuDF Version:', cudf.__version__)


df = cudf.from_pandas(pandas_df)
# df = cudf.DataFrame.from_pandas(pandas_df)  # alternative
print(df)

cuDF Version: 0.10.0a+1601.gd910060a9
     a     b
0    0   0.0
1    1   0.1
2    2   0.2
3    3  null
4    4   0.4
5    5   0.5
6    6   0.6
7    7   0.7
8    8   0.8
9    9   0.9
10  10   1.0


In [10]:
ddf = dask_cudf.from_cudf(df, npartitions=8)
ddf

,a,b
npartitions=5,,
0,int64,float64
2,...,...
...,...,...
8,...,...
10,...,...


#### Inspecting a Dask cuDF DataFrame (coming soon!)

In [11]:
ddf

,a,b
npartitions=5,,
0,int64,float64
2,...,...
...,...,...
8,...,...
10,...,...


In [12]:
print(ddf)

<dask_cudf.DataFrame | 5 tasks | 5 npartitions>


In [13]:
print(ddf.compute())

     a     b
0    0   0.0
1    1   0.1
2    2   0.2
3    3  null
4    4   0.4
5    5   0.5
6    6   0.6
7    7   0.7
8    8   0.8
9    9   0.9
10  10   1.0


In [14]:
print(type(ddf.compute()))

<class 'cudf.core.dataframe.DataFrame'>


In [15]:
type(ddf)

dask_cudf.core.DataFrame

In [16]:
ddf.npartitions

5

<a id="io"></a>
## Input/Output (coming soon!)

#### Writing and Loading CSV Files

<a id="daskcudfapi"></a>
## Dask cuDF API

#### Selecting Rows or Columns

In [22]:
df = cudf.DataFrame({'a': np.arange(0, 100).astype(np.int64), 
                     'b': np.arange(100, 0, -1).astype(np.float32), 
                     'c': np.arange(100, 200).astype(np.float32)})
ddf = dask_cudf.from_cudf(df, npartitions=8)

In [28]:
ddf.iloc[:, 1]

<dask_cudf.Series | 16 tasks | 8 npartitions>

In [27]:
ddf.iloc[:, ['a']]

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [18]:
ddf.iloc[:, ['a', 'b']]

,a,b
npartitions=8,,
0,float32,float32
13,...,...
...,...,...
91,...,...
99,...,...


#### Dropping Rows or Columns (coming soon!)

In [19]:
df = cudf.DataFrame({'a': np.arange(0, 100).astype(np.float32), 
                     'b': np.arange(100, 0, -1).astype(np.float32), 
                     'c': np.arange(100, 200).astype(np.float32)})
ddf = dask_cudf.from_cudf(df, npartitions=8)

In [20]:
# ddf.drop('a', axis=1)

#### Defining New Columns (coming soon!)

In [21]:
df = cudf.DataFrame({'a': np.arange(0, 100).astype(np.float32), 
                     'b': np.arange(100, 0, -1).astype(np.float32), 
                     'c': np.arange(100, 200).astype(np.float32)})
ddf = dask_cudf.from_cudf(df, npartitions=8)

In [22]:
# ddf['d'] = 

#### Missing Data  (coming soon!)

In [23]:
df = cudf.DataFrame({'a': [0, None, 2, 3, 4, 5, 6, 7, 8, None, 10],
                     'b': [0.0, 0.1, 0.2, None, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 
                     'c': [0.0, 0.1, None, None, 0.4, 0.5, None, 0.7, 0.8, 0.9, 1.0]})
ddf = dask_cudf.from_cudf(df, npartitions=8)

In [24]:
print(ddf.compute())

   a    b    c
0  0  0.0  0.0
1     0.1  0.1
2  2  0.2     
3  3          
4  4  0.4  0.4
5  5  0.5  0.5
6  6  0.6     
7  7  0.7  0.7
8  8  0.8  0.8
9     0.9  0.9
[1 more rows]


In [25]:
new_ddf = ddf.fillna(-1)

In [26]:
print(new_ddf.compute())

    a     b     c
0   0   0.0   0.0
1  -1   0.1   0.1
2   2   0.2  -1.0
3   3  -1.0  -1.0
4   4   0.4   0.4
5   5   0.5   0.5
6   6   0.6  -1.0
7   7   0.7   0.7
8   8   0.8   0.8
9  -1   0.9   0.9
[1 more rows]


#### Boolean Indexing (coming soon!)

In [27]:
df = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                     'b': np.random.randint(2, size=100).astype(np.int32), 
                     'c': np.arange(0, 100).astype(np.int32), 
                     'd': np.arange(100, 0, -1).astype(np.int32)})
ddf = dask_cudf.from_cudf(df, npartitions=8)

In [28]:
mask = ddf['a'] == 2
subset = ddf[mask]

In [29]:
subset.compute()

<cudf.DataFrame ncols=4 nrows=25 >

#### Sorting Data (coming soon!)

In [30]:
df = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                     'b': np.random.randint(2, size=100).astype(np.int32), 
                     'c': np.arange(0, 100).astype(np.int32), 
                     'd': np.arange(100, 0, -1).astype(np.int32)})
ddf = dask_cudf.from_cudf(df, npartitions=8)

In [31]:
result = ddf.sort_values('d').compute()
print(result.head())

   a  b   c  d
0  3  0  99  1
1  3  0  98  2
2  3  0  97  3
3  3  0  96  4
4  3  1  95  5


In [32]:
# result = ddf.sort_values('c', ascending=False).compute()
# print(result.head())

In [33]:
result = ddf.sort_values(['a', 'b']).compute()
print(result.head())

   a  b  c   d
0  0  0  1  99
1  0  0  3  97
2  0  0  4  96
3  0  0  5  95
4  0  0  6  94


In [34]:
# result = ddf.sort_values(['a', 'b'], ascending=False).compute()
# print(result.head())

In [35]:
# result = ddf.sort_values(['a', 'b'], ascending=[False, True]).compute()
# print(result.head())

#### Statistical Operations (coming soon!)

In [36]:
df = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                     'b': np.random.randint(2, size=100).astype(np.int32), 
                     'c': np.arange(0, 100).astype(np.int32), 
                     'd': np.arange(100, 0, -1).astype(np.int32)})
ddf = dask_cudf.from_cudf(df, npartitions=8)

In [37]:
ddf['a'].sum().compute()

150

In [38]:
# ddf.sum().compute()

#### Histogramming (coming soon!)

In [39]:
df = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                     'b': np.random.randint(2, size=100).astype(np.int32), 
                     'c': np.arange(0, 100).astype(np.int32), 
                     'd': np.arange(100, 0, -1).astype(np.int32)})
ddf = dask_cudf.from_cudf(df, npartitions=8)

In [40]:
result = ddf['a'].value_counts().compute()
print(result)

0    25
1    25
2    25
3    25
dtype: int64


#### Concatenations (coming soon!)

In [41]:
df1 = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                      'b': np.random.randint(2, size=100).astype(np.int32), 
                      'c': np.arange(0, 100).astype(np.int32), 
                      'd': np.arange(100, 0, -1).astype(np.int32)})
df2 = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                      'b': np.random.randint(2, size=100).astype(np.int32), 
                      'c': np.arange(0, 100).astype(np.int32), 
                      'd': np.arange(100, 0, -1).astype(np.int32)})
ddf1 = dask_cudf.from_cudf(df1, npartitions=8)
ddf2 = dask_cudf.from_cudf(df2, npartitions=8)

In [42]:
ddf = dask_cudf.concat([ddf1, ddf2], axis=0)
ddf.compute()

<cudf.DataFrame ncols=4 nrows=200 >

In [43]:
# df1 = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
#                       'b': np.random.randint(2, size=100).astype(np.int32), 
#                       'c': np.arange(0, 100).astype(np.int32), 
#                       'd': np.arange(100, 0, -1).astype(np.int32)})
# df2 = cudf.DataFrame({'e': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
#                       'f': np.random.randint(2, size=100).astype(np.int32), 
#                       'g': np.arange(0, 100).astype(np.int32), 
#                       'h': np.arange(100, 0, -1).astype(np.int32)})
# ddf1 = dask_cudf.from_cudf(df1, npartitions=8)
# ddf2 = dask_cudf.from_cudf(df2, npartitions=8)

In [44]:
# ddf = dask_cudf.concat([ddf1, ddf2], axis=1)
# ddf.compute()

#### Joins (coming soon!)

In [45]:
df1 = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                      'b': np.random.randint(2, size=100).astype(np.int32), 
                      'c': np.arange(0, 100).astype(np.int32), 
                      'd': np.arange(100, 0, -1).astype(np.int32)})
df2 = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                      'b': np.random.randint(2, size=100).astype(np.int32), 
                      'e': np.arange(0, 100).astype(np.int32), 
                      'f': np.arange(100, 0, -1).astype(np.int32)})
ddf1 = dask_cudf.from_cudf(df1, npartitions=8)
ddf2 = dask_cudf.from_cudf(df2, npartitions=8)

In [46]:
ddf1.merge(ddf2, on=['a'])

,a,b_x,c,d,b_y,e,f
npartitions=8,,,,,,,
,int32,int32,int32,int32,int32,int32,int32
,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


In [47]:
ddf1.merge(ddf2, on=['a', 'b'])

,a,b,c,d,e,f
npartitions=8,,,,,,
,int32,int32,int32,int32,int32,int32
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [48]:
dask_cudf.DataFrame.merge(ddf1, ddf2, on=['a'])

,a,b_x,c,d,b_y,e,f
npartitions=8,,,,,,,
,int32,int32,int32,int32,int32,int32,int32
,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


In [49]:
dask_cudf.DataFrame.merge(ddf1, ddf2, on=['a', 'b'])

,a,b,c,d,e,f
npartitions=8,,,,,,
,int32,int32,int32,int32,int32,int32
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


#### Appends (coming soon!)

#### Groupbys (coming soon!)

In [50]:
df = cudf.DataFrame({'a': np.repeat([0, 1, 2, 3], 25).astype(np.int32), 
                     'b': np.random.randint(2, size=100).astype(np.int32), 
                     'c': np.arange(0, 100).astype(np.int32), 
                     'd': np.arange(100, 0, -1).astype(np.int32)})
ddf = dask_cudf.from_cudf(df, npartitions=8)

In [51]:
result = ddf.groupby('a').sum().compute()
print(result)

    b     c     d
0  13   300  2200
1  16   925  1575
2  10  1550   950
3  15  2175   325


In [52]:
result = ddf.groupby(['a', 'b']).sum().compute().to_pandas()
print(result)

        c     d
a b            
0 0   138  1062
  1   162  1138
1 0   331   569
  1   594  1006
2 0   917   583
  1   633   367
3 0   881   119
  1  1294   206


#### One Hot Encoding (coming soon!)

<a id="conclusion"></a>
## Conclusion

In this notebook, we showed how to work with cuDF DataFrames distributed across multiple GPUs using Dask.

To learn more about RAPIDS, be sure to check out: 

* [Open Source Website](http://rapids.ai)
* [GitHub](https://github.com/rapidsai/)
* [Press Release](https://nvidianews.nvidia.com/news/nvidia-introduces-rapids-open-source-gpu-acceleration-platform-for-large-scale-data-analytics-and-machine-learning)
* [NVIDIA Blog](https://blogs.nvidia.com/blog/2018/10/10/rapids-data-science-open-source-community/)
* [Developer Blog](https://devblogs.nvidia.com/gpu-accelerated-analytics-rapids/)
* [NVIDIA Data Science Webpage](https://www.nvidia.com/en-us/deep-learning-ai/solutions/data-science/)
